[![Open holes - Construction safety](https://www.keithconstruction.ca/wp-content/uploads/2020/09/Construction-Safety-1024x536.png)](https://github.com/BitSpaceDevelopment/CV-Construction-safety-model)

# Train a pre-trained Faster R-CNN model using OpenCV (Open Source Computer Vision Library), Pytorch deep learning framework and Robotflow for data preparation

---
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/BitSpaceDevelopment/CV-Construction-safety-model)
## Use case: Open hole detection
Link to original data: https://app.roboflow.com/northeastern-4sfxe/open-hole-detection-construction-safety/1


 This model will be deployed to our Jetson-based edge inference systems for job sites allowing us to monitor and report on unsafe conditions surrounding excavation and earth moving. This model should run on a single-board computer; we will likely combine functionality with the PPE model to determine if somebody is wearing a harness near an unsafe excavation. 

---
## About the model:

 The Faster R-CNN model represents a significant advancement in object detection by combining region proposal techniques and deep learning. Its two-stage framework, incorporating a region proposal network and a region-based CNN, has demonstrated remarkable accuracy and efficiency, making it a widely adopted choice for object detection tasks across different domains.
 
---
## Note: Data download

Faster R-CNN has achieved state-of-the-art performance on various benchmark datasets, such as COCO (Common Objects in Context) and PASCAL VOC (Visual Object Classes); therefore, if we use Robotflow to download data, we can export data under **Pascal VOC format**.
 
 
---

## Pro Tip: Use GPU Acceleration

If you are running this notebook in Google Colab, navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`. This will ensure your notebook uses a GPU, which will significantly speed up model training times.

---

## Steps in this training

In this notebook, we are going to cover:

- Set GPU
- Install YOLOv7
- Preparing a custom dataset: Upload, annotate and prepared on Roboflow
- Custom Training
- Validate Custom Model
- Inference with Custom Model
- Download weights
- Deploy models

Visualization of workflow:
![](https://2486075003-files.gitbook.io/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2F-M6S9nPJhEX9FYH6clfW%2Fuploads%2FfHpPTWNdCVR9qHQDeskF%2FScreen%20Shot%202022-08-24%20at%2012.35.36%20PM.png?alt=media&token=623927fe-3099-4ccd-8aaa-890bf5c0b03b)
**Let's begin!**
---
---

## 0. Download and Import libraries

In [ ]:
!pip install torchvision
!pip install roboflow
# Clear the download logs
from IPython import display
display.clear_output()

In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import VOCDetection
from torchvision.transforms import transforms
from PIL import Image
import os
import xml.etree.ElementTree as ET

## 1. Set up

In [ ]:
# Define the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Check if we have the access to GPU
!nvidia-smi

Wed May 17 23:32:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. Data download

In [ ]:
# Define the Roboflow API key
api_key = "bvgx6NFEHR0zZurL8c9w"

# Initialize Roboflow
rf = Roboflow(api_key=api_key)

# Specify the Roboflow project and dataset version
workspace = "northeastern-4sfxe"
project_name = "open-hole-detection-construction-safety"
version = 3

# Download the dataset from Roboflow
dataset = rf.workspace(workspace).project(project_name).version(version).download("voc")


NameError: ignored

In [ ]:
# Function to parse the XML annotation file and extract bounding box coordinates
def parse_annotation(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    boxes = []
    labels = []

    for obj in root.findall('object'):
        label = obj.find('name').text
        labels.append(label)

        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        boxes.append([xmin, ymin, xmax, ymax])

    return {'boxes': boxes, 'labels': labels}

In [ ]:
# Define the transformation to apply to the images
transform = transforms.Compose([transforms.ToTensor()])


# Define the dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_dir, transform=None):
        self.images_dir = images_dir
        self.annotations_dir = annotations_dir
        self.transform = transform
        self.images = sorted(os.listdir(images_dir))
        self.annotations = sorted(os.listdir(annotations_dir))

    def __getitem__(self, index):
        image_path = os.path.join(self.images_dir, self.images[index])
        annotation_path = os.path.join(self.annotations_dir, self.annotations[index])
        
        image = Image.open(image_path).convert("RGB")
        
        # Parse the XML annotation file and extract bounding box coordinates
        annotation_data = parse_annotation(annotation_path)
        boxes = annotation_data['boxes']
        labels = annotation_data['labels']
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, {"boxes": boxes, "labels": labels}

    def __len__(self):
        return len(self.images)


In [ ]:

# Specify the paths to the downloaded dataset
images_dir = "Open-hole-Detection---Construction-Safety-3/train"
annotations_dir = "Open-hole-Detection---Construction-Safety-3/train"

In [ ]:
# Create the dataset
dataset = CustomDataset(images_dir, annotations_dir, transform=transform)

In [ ]:
# Create the data loader
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)


In [ ]:
# Load the pre-trained model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
# Define the optimizer and learning rate scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


In [ ]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    # Update learning rate
    lr_scheduler.step()

    # Print training progress
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses.item():.4f}")

print("Training finished!")

UnidentifiedImageError: ignored

In [ ]:

# Load the pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)

# Define the optimizer and learning rate scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
    lr_scheduler.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses.item()}")

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "path/to/save/model.pth")
